In [120]:
import os
import pandas as pd
import numpy as np
import time
import random
import datetime
import matplotlib.pyplot as plt
import scipy.optimize as spo

In [140]:
def symbol_to_path(symbol, base_dir='C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data'):
    """Returns the file path given the symbol"""
    return os.path.join(base_dir, '{}.csv'.format(str(symbol)))


def normalize_data(df):
    '''Normalize stock prices to have initial value equal to 1'''
    return df/df.ix[0,:]

In [64]:
def get_data(symbols, dates, col='Adj Close'):
    """"For the given sybols, returns the values of cols in the range dates"""
    df = pd.DataFrame(index=dates)
    if 'SPY' not in symbols:
        symbols.insert(0, 'SPY')
    
    for s in symbols:
        df_temp = pd.read_csv(symbol_to_path(s), index_col='Date', parse_dates=True, usecols=['Date', col],
                         na_values=['nan'])
    
        df_temp = df_temp.rename(columns={col:s})
        df = df.join(df_temp)
        
        if s == 'SPY':
            df = df.dropna(subset=['SPY'])
        
#    df.fillna(method='ffill', inplace=True)
#    df.fillna(method='bfill', inplace=True)
    
    return df


def plot_data(df, title='Stock prices'):
    '''Plot stock prices in df, with title'''
    ax = df.plot(title=title, fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
  
    return ax


def plot_selected(df, columns, start_index, end_index, title='Stock prices'):
    '''Plot desired columns, in desired range'''
    ax = df.ix[start_index:end_index, columns].plot(title=title, fontsize=10)
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    
    plt.show()

In [4]:
def get_bollinger_bands(rm, rstd):
    upper = rm + 2*rstd
    lower = rm - 2*rstd
    
    return upper, lower


def get_daily_returns(df):
    dr = df.copy()
    dr = (df/ df.shift(1)) - 1
    dr.ix[0,:] = 0
    
    return dr

In [38]:
class Portfolio:
    def __init__(self, start_date=datetime.date.today(), end_date=datetime.date.today(), symbols=None, allocation=None):
        """Portfolio object. It is initialized by a list of symbols, that represents the stocks it has a partecipation; 
        allocation should be a dict, with keys equal to symbols"""
        
        #Initialization of the portfolio values. 'RiFr' stands for the risk free part, where all unused allocations goes.
        self.symbols = symbols + ['RiFr']
        self.allocation = dict()
        for sym in symbols:
            if allocation.get(sym) is not None:
                self.allocation[sym] = allocation[sym]
        self.allocation['RiFr'] = 1 - sum(allocation.values())
        
        #valid value checks if the portfolio is valid. First check if the allocations sums at most one
        self.valid = True
        if self.allocation['RiFr'] < 0:
            print('Portfolio allocation value invalid: sum of allocations greater than one')
            self.valid = False
        
        #Initialize the stocks data of the portfolio
        self.dates = pd.date_range(start_date, end_date)
        self.data_frame = get_data(symbols, self.dates)
        self.data_frame = normalize_data(self.data_frame)
        
        #Add to the data_frame the risk free lending interest, that for the moment we assume = 1
        self.risk_free_value = pd.DataFrame(index=self.dates, columns=['RiFr'])
        self.risk_free_value['RiFr'] = 1
        self.data_frame = self.data_frame.join(self.risk_free_value, how='inner')
        
        #computes the value of the portfolio
        self.value = pd.DataFrame(index=self.dates, columns=['Value'])
        self.value['Value'] = 0
        for s in self.symbols:
            if self.allocation.get(s) is not None:
                self.value['Value'] += self.allocation[s] * self.data_frame[s]
        self.data_frame = self.data_frame.join(self.value, how='inner')

        #computes the daily returns of the variables
        self.daily_returns = get_daily_returns(self.data_frame)
        
    
    def get_sharp_ratio(self, daily_rfrate = 0):
        """Computes the sharp ratio of the portfolio, over the whole period"""
        return  np.sqrt(252) * (self.daily_returns['Value'] - daily_rfrate).mean()  / self.daily_returns['Value'].std()
    
    
    def get_mean_daily_returns(self):
        """Computes the mean of the daily return of the portfolio, over the whole period"""
        return self.daily_returns['Value'].mean()
    
    
    def get_cumulative_returns(self):
        """Computes the cumulative return of the portfolio, over the wole period"""
        return self.data_frame['Value']
    
    
    def get_risk(self):
        """Computes the risk of the portfolio, over the whole period"""
        return self.daily_returns['Value'].std()
    
    
    def plot_data_now(self):
        """Plots the underlying assets"""
        plot_data(self.data_frame)
        plt.show()
        
    
    def plot_daily_returns_now(self, col=['Value']):
        """Plots the daily returns of the underlying"""
        plot_data(self.daily_returns[col])
        plt.show()
        

In [18]:
def maximize_sharp_ratio(s_date, e_date, lsymbols):
    """Maximize a portfolio on assets given by symbols, between dates, w.r.t. the sharp ratio"""
    n_stocks = len(lsymbols)
    alloc0 = np.zeros(n_stocks)
    
    for s in range(n_stocks):
        alloc0[s] = random.random()/n_stocks
    
    def con(X):
        return 0.999 - X.sum() 
    def cos_pos(X):
        return X - 0.0001
    cons = [{'type': 'ineq', 'fun': con}, {'type': 'ineq', 'fun': cos_pos}]
    
    def sr(X):
        loc_alloc = dict()
        for s in lsymbols:
            loc_alloc[s] = X[lsymbols.index(s)]
        pf = Portfolio(s_date, e_date, lsymbols, loc_alloc)
        print(pf.get_sharp_ratio())
        return - pf.get_sharp_ratio()
    
    result = spo.minimize(sr, alloc0, constraints=cons, method='SLSQP')
    
    return result


In [116]:
def get_symbols_from_txt(filename):
    """Read a list of symbols from a file to a list"""
    f = open(filename)
    sy = f.read().splitlines()
    f.close()
    
    return sy


def check_file(file_name):
    exists = False
    try:
        f = open(file_name)
        exists = True
        f.close()
    except FileNotFoundError:
        print('File ', file_name, 'not found. ')
    return exists


def check_symbol_list(lsymbols):
    for s in lsymbols:
        file_n = symbol_to_path(s)
        s_exists = check_file(file_n)
        if not s_exists:
            lsymbols.remove(s)
    
    return lsymbols

In [129]:
s_date = '2015-01-01'
e_date = '2015-12-31'

In [131]:
symboloni = check_symbol_list(get_symbols_from_txt('C:/PyExercises/SP500-symbols.txt'))

File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\CFN.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\COV.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\DTV.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\EP.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\FDO.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\HSP.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\HCBK.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\TEG.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\JDSU.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\JOYG.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data2\KFT.csv not found. 
File  C:/Users/Pietro T/Documenti/ML_and_finance/exercises/data

In [137]:
symboloni = check_symbol_list(symboloni)

In [138]:
ddf = get_data(symboloni, pd.date_range(s_date, e_date))

In [139]:
print(ddf)

                   SPY         MMM         ACE        ABT        ANF  \
2015-01-02  201.276557  158.704642  114.440002  43.691606  27.394057   
2015-01-05  197.641575  155.125423  112.570000  43.701335  27.096921   
2015-01-06  195.779996  153.471236  111.580002  43.205062  27.681607   
2015-01-07  198.219643  154.583705  113.169998  43.555370  28.112935   
2015-01-08  201.737056  158.288686  115.000000  44.450612  27.902065   
2015-01-09  200.120422  156.344288  113.370003  43.983531  27.020241   
2015-01-12  198.552765  155.493024  112.000000  44.353305  27.116091   
2015-01-13  197.994297  155.376931  112.169998  43.639192  27.010655   
2015-01-14  196.798962  154.622393  109.849998  43.316372  27.173601   
2015-01-15  194.996167  154.448276  109.820000  42.993555  25.793354   
2015-01-16  197.553398  156.711889  111.750000  43.521804  25.764598   
2015-01-20  197.974697  156.653850  110.470001  43.257680  24.834850   
2015-01-21  198.974079  157.234255  110.540001  43.101162  25.15

In [125]:
whos

Variable               Type              Data/Info
--------------------------------------------------
Portfolio              type              <class '__main__.Portfolio'>
UU2                    OptimizeResult         nit: 7\n  status: 0\<...>terminated successfully.'
allocco                dict              n=11
checK_file             function          <function checK_file at 0x0000003CB7C9A620>
check_file             function          <function check_file at 0x0000003CB7212E18>
check_symbol_list      function          <function check_symbol_li<...>st at 0x0000003CB54AC488>
datetime               module            <module 'datetime' from '<...>onda3\\lib\\datetime.py'>
df                     DataFrame                            SPY   <...>\n[12 rows x 464 columns]
e_date                 str               2008-12-31
get_bollinger_bands    function          <function get_bollinger_b<...>ds at 0x0000003CB54B7730>
get_daily_returns      function          <function get_daily_retur<...>ns a